In [1]:
%pip install flask-apscheduler

Note: you may need to restart the kernel to use updated packages.


c:\Users\amogh\Desktop\NexTurn\timeslot_app\.venv\Scripts\python.exe: No module named pip


In [4]:
from datetime import datetime
import psycopg2
import os

def get_db_connection():
    conn = psycopg2.connect(
        host='localhost',
        dbname=os.getenv('POSTGRES_DB'),
        user=os.getenv('POSTGRES_USER'),
        password=os.getenv('POSTGRES_PASSWORD')
    )
    return conn

def slot_not_inside(slots1, slots2):
    for i in range(7):
        if not (slots2[i][0] <= slots1[i][0] <= slots1[i][1] <= slots2[i][1]):
            return True
    return False

def slot_overlap(slots1, slots2):
    for i in range(7):
        if (slots1[i][0] == -1 and slots1[i][1] == -1) or (slots2[i][0] == -1 and slots2[i][1] == -1):
            continue
        if (slots2[i][0] <= slots1[i][0] <= slots2[i][1]) or (slots2[i][0] <= slots1[i][1] <= slots2[i][1]):
            return True
    return False

def date_overlap(start_date1, end_date1, start_date2, end_date2):
    return start_date1 <= end_date2 and start_date2 <= end_date1

def conflict(StartDate: str, EndDate: str, HoursBooked: list, WorkerName: str) -> bool:
    """Function to check for conflicts in bookings"""
    for h in range(len(HoursBooked)):
        if len(HoursBooked[h]) == 0:
            HoursBooked[h] = [-1, -1]
    conn = get_db_connection()
    cur = conn.cursor()
    cur.execute("Select AvailableHours from availability where WorkerName = %s;", (WorkerName,))
    availability = cur.fetchone()[0]
    
    if len(availability) != 0:
        if slot_not_inside(HoursBooked, availability):
            return True
    cur.execute("SELECT StartDate, EndDate, HoursBooked from bookings where WorkerName = %s;", (WorkerName,))
    bookings = cur.fetchall()
    if len(bookings) != 0:
        StartDate = datetime.strptime(StartDate, '%Y%m%d').date()
        EndDate = datetime.strptime(EndDate, '%Y%m%d').date()
        for b in bookings:
            if date_overlap(StartDate, EndDate, b[0], b[1]) and slot_overlap(HoursBooked, b[2]):
                print("Conflict with booking:", b, StartDate, EndDate, HoursBooked)
                return True
    return False

def hourly_bid_accept() -> str:
    conn = get_db_connection()
    cur = conn.cursor()
    cur.execute("SELECT username from users where usertype='worker';")
    workers = cur.fetchall()
    print(workers)

In [5]:
hourly_bid_accept()

[('abc',)]
